# A place to store functions that are useful
Cassie Lumbrazo

## Import Packages

In [2]:
# import packages 
%matplotlib widget
# plotting packages 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 

# interactive plotting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# data packages 
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime

import csv 
import copy 
import os.path 

## Useful code

In [ ]:
met = met.loc['2020-11-01T00:00:00':'2021-05-18T00:00:00'] 

## Functions

In [1]:
# Calculating the specific humidtiy from RH, temperature, and airpressure
def spechum(rh, T, p):
    T0 = 273.15
    return rh * np.exp((17.67*(T-T0))/(T-29.65)) / (0.263*p)

# Example code: how I used it  
#niwot_spechum = spechum(ds.RH, ds.temp_K, ds.air_press_Pa)

In [ ]:
# Calculating the RH from specific humidtiy, temperature, and air pressure
def RH(spechum, T, p):
    T0 = 273.15
    return  (np.exp((17.67*(T-T0))/(T-29.65)) / (0.263*p)) / spechum

# Example code: how I used it  
#niwot_RH = RH(ds.spechum, ds.temp_K, ds.air_press_Pa)

In [4]:
# Working with HRRR downloaded data 
def getPixelCoords(dataArrayIn,lat,lon,pointLat,pointLon):
    from scipy import spatial
    pixel_center_points = np.stack((dataArrayIn[lat].values.flatten(), dataArrayIn[lon].values.flatten()), axis=-1)
    tree = spatial.KDTree(pixel_center_points)
    searchCoords = (pointLat,pointLon)
    distance, index = tree.query(searchCoords)
    pixel_coords_Out = np.unravel_index(index, dataArrayIn.shape)
    return pixel_coords_Out

# Example code: how I used it  
# mw_index = getPixelCoords(ds['t2m'],'latitude','longitude', mw_lat, mw_lon)

In [8]:
# Calculating Wind Speed from U and V vectors, needed this for hrrrpy 
def calWindSpeed(u, v):
    u = ds['u10'][:,index[1],index[2]]
    v = ds['v10'][:,index[1],index[2]]

    windspd = np.sqrt(np.square(u)+np.square(v))
    
    return windspd 

In [1]:
# Shading the snow in the canopy classifications
def Shade(fig, df, variable, level, mode, fillcolor, layer, opacity):
  
    """
    Set a specified color as background for given levels of a specified variable using a shape.
    
    Keyword arguments:
    ==================
    fig -- plotly figure
    variable -- column name in a pandas dataframe
    level -- int or float
    mode -- set threshold above or below
    fillcolor -- any color type that plotly can handle
    layer -- position of shape in plotly fiugre, like "below"
    opacity -- opacity of the fillcolor, WORKING but the ordering matters.. so pay attention to that 
    """
    
    if mode == 'above':
        m = df[variable].gt(level) # greater than function
    
    if mode == 'below':
        m = df[variable].lt(level) # less than function
        
    df1 = df[m].groupby((~m).cumsum())['datetime'].agg(['first','last'])

    for index, row in df1.iterrows():
        # print(row['first'], row['last'])
        fig.add_shape(type="rect", xref="x", yref="paper", x0=row['first'], y0=0, x1=row['last'], y1=1, opacity=opacity, line=dict(color="rgba(0,0,0,0)",width=3),fillcolor=fillcolor,layer=layer) 
        fig.add_shape(type="rect", xref="x", yref="paper", x0=row['first'], y0=0, x1=row['last'], y1=1, opacity=opacity, line=dict(color="rgba(0,0,0,0)",width=3),fillcolor=fillcolor,layer=layer)
   
    return(fig)

In [ ]:
# Function for calculating F-score and BA

def getStats(smelt):
    # Saving the variables 
    smelt_b = (smelt.output.scalarCanopyWat_mean).squeeze('hru')
    #smelt_b = (smelt.scalarCanopyWat_mean).squeeze('hru')

    min_value = 0.1
    #min_value = 2
    # Making the variables binary, using a threshold for that datetime 
    issuemelt    = smelt_b.where(smelt_b > min_value, 0)         # change any value less than 2, to 0 --- 2 or whatever SWE canopy height
    smelt_binary = issuemelt.where(issuemelt <= min_value, 1)     # change any value greater than 2, to 1
    smelt_binary = smelt_binary.to_dataframe()

    # Create Fixed Daterange and index by it
    smelt_binary['datetime'] = pd.date_range(start='2016-10-10 06:00:00', end='2017-05-16T06:00:00',freq='H')
    smelt_binary = smelt_binary.set_index('datetime')

    dfmelt = obs
    dfmelt = dfmelt.loc['2016-10-10T06:00:00':'2017-05-16T06:00:00'] 
    dfmelt['model'] = smelt_binary['scalarCanopyWat_mean']

    #dfmelt = dfmelt.append(smelt_binary['scalarCanopyWat_mean'])

    # True Negative and Postive
    mtruenegdf = dfmelt.where((dfmelt['model'] == 0) & (dfmelt['value'] == 0)) # where this condition is true, keep array and replace else with nan
    mtrueposdf = dfmelt.where((dfmelt['model'] == 1) & (dfmelt['value'] == 1))

    # False Negative and Postive 
    mfalsenegdf = dfmelt.where((dfmelt['model'] == 0) & (dfmelt['value'] == 1))
    mfalseposdf = dfmelt.where((dfmelt['model'] == 1) & (dfmelt['value'] == 0))

    # Add them up to see how many there are
    mTN = mtruenegdf.count()
    mTP = mtrueposdf.count()
    mFN = mfalsenegdf.count()
    mFP = mfalseposdf.count()

    # Calculate F-Score
    mfscore = (2*mTP)/((2*mTP) + mFP + mFN)

    # Balanced Accuracy
    mBA = .5*((mTP/(mTP+mFN))+(mTN/(mTN+mFP)))

    # Print everything 
    #print("MELT RESULTS")
    #print('F-Score:', mfscore.value)
    #print('Balanced Accuracy:', mBA.value)
    print(mBA.value, mTP.value, mTN.value, mFP.value, mFN.value)
    return 